## Hawk/Dove Analysis (single risk attitude)

All agents with the same risk attitude.

Analysis includes:
- simulation length and convergence
- % hawk by risk level
- points by risk level

In [29]:
import pandas as pd
#df = pd.read_csv("../hawkdove_2023-09-25T150643_380857.csv")
# more recent run that includes data collection for agent cumulative points
df = pd.read_csv("../../data/hawkdove_risk-single_2023-09-27T154226_942256.csv")
# smaller version - only 200 rounds
# df = pd.read_csv("../../data/hawkdove_risk-single_2023-09-27T175109_555307.csv")


In [30]:
df.head()

,RunId,iteration,Step,grid_size,risk_attitudes,agent_risk_level,max_agent_points,percent_hawk,AgentID,risk_level,choice,points
0,0,0,0,20,single,0,21.0,0.4875,NaN,NaN,NaN,NaN
1,0,0,1,20,single,0,42.0,0.1950,0.0,0.0,hawk,9.0
2,0,0,1,20,single,0,42.0,0.1950,1.0,6.0,hawk,12.0
3,0,0,1,20,single,0,42.0,0.1950,2.0,5.0,dove,11.3
4,0,0,1,20,single,0,42.0,0.1950,3.0,6.0,hawk,18.0


In [31]:
model_df = df[df.AgentID.isna()]
model_df.head()

,RunId,iteration,Step,grid_size,risk_attitudes,agent_risk_level,max_agent_points,percent_hawk,AgentID,risk_level,choice,points
0,0,0,0,20,single,0,21.0,0.4875,NaN,NaN,NaN,NaN
400001,1,0,0,20,single,1,24.0,0.4775,NaN,NaN,NaN,NaN
800002,2,0,0,20,single,2,21.0,0.5300,NaN,NaN,NaN,NaN
1200003,3,0,0,20,single,3,24.0,0.5100,NaN,NaN,NaN,NaN
1600004,4,0,0,20,single,4,21.0,0.5075,NaN,NaN,NaN,NaN


In [32]:
# get model-level data across all rounds and runs
run_df = df[['RunId', 'iteration', 'Step', 'agent_risk_level', 'percent_hawk']]

In [33]:
run_df = run_df.drop_duplicates()
run_df

,RunId,iteration,Step,agent_risk_level,percent_hawk
0,0,0,0,0,0.4875
1,0,0,1,0,0.1950
401,0,0,2,0,0.6425
801,0,0,3,0,0.4925
1201,0,0,4,0,0.2850
...,...,...,...,...,...
3598009,8,0,996,8,0.0000
3598409,8,0,997,8,0.0000
3598809,8,0,998,8,0.0000
3599209,8,0,999,8,0.0000


In [34]:
import altair as alt

alt.data_transformers.disable_max_rows()

# alt.Chart(run_df[run_df.Step < 100]).mark_line().encode(
alt.Chart(run_df).mark_line().encode(    
    x='Step',
    y='percent_hawk',
    color='agent_risk_level:N',
).properties(
    width=800,
    height=300
)

alt.Chart(...)

In [35]:
runzero = run_df[run_df.RunId == 0]
runzero

,RunId,iteration,Step,agent_risk_level,percent_hawk
0,0,0,0,0,0.4875
1,0,0,1,0,0.1950
401,0,0,2,0,0.6425
801,0,0,3,0,0.4925
1201,0,0,4,0,0.2850
...,...,...,...,...,...
398001,0,0,996,0,0.4775
398401,0,0,997,0,0.3525
398801,0,0,998,0,0.5450
399201,0,0,999,0,0.4775


In [36]:
run_one = run_df[run_df.RunId == 1]


In [37]:
run_zero_chart = alt.Chart(runzero[runzero.Step < 150]).mark_line().encode(
    x='Step', # alt.X('Step', scale=alt.Scale(domain=[0, 1])),
    y='percent_hawk',
    # color='agent_risk_level:N',
).properties(
    width=800,
    height=300
)
run_zero_chart

alt.Chart(...)

In [38]:
# how to work with this oscillating pattern of alternating hawks?
# can we use a rolling mean?

line = alt.Chart(runzero).mark_line(
    color='red',
    size=3
).transform_window(
    rolling_mean='mean(percent_hawk)',
    frame=[-15, 15]
).encode(
    x='Step',
    y='rolling_mean:Q'
).properties(
    width=800,
    height=300
)

points = alt.Chart(runzero).mark_line().encode(
    x='Step',
    y='percent_hawk'
)

points + line


alt.LayerChart(...)

In [39]:
# create and display charts for each run / risk level

charts = []

for i in range(8):
    run_i = run_df[run_df.RunId == i]
    risk_level = run_i.agent_risk_level.unique()[0]
    run_chart = alt.Chart(run_i).mark_line().encode(
        x='Step',
        y=alt.Y('percent_hawk', scale=alt.Scale(domain=[0, 1.0]))
        # color='agent_risk_level:N',
    ).properties(
        # title=f'Run {i}, risk level {risk_level}',
        title=f'Risk level {risk_level}',
        width=600,
        height=90
    )
    charts.append(run_chart)

combined_chart = None
for c in charts:
    if combined_chart is None:
        combined_chart = c
    else:
        combined_chart = alt.vconcat(combined_chart, c)

combined_chart

alt.VConcatChart(...)

In [40]:
# do the same thing, but display beginning instead of end and add the rolling mean

rollmean_charts = []

for i in range(8):
    run_i = run_df[run_df.RunId == i]
    risk_level = run_i.agent_risk_level.unique()[0]
    run_chart = alt.Chart(run_i).mark_line().encode(
        x='Step',
        y=alt.Y('percent_hawk', scale=alt.Scale(domain=[0, 1.0]))
        # color='agent_risk_level:N',
    ).properties(
        # title=f'Run {i}, risk level {risk_level}',
        title=f'Risk level {risk_level}',
        width=800,
        height=90
    )
    rollmean_line = alt.Chart(runzero[runzero.Step < 300]).mark_line(
        color='red',
        size=3
    ).transform_window(
        rolling_mean='mean(percent_hawk)',
        frame=[-15, 15]
    ).encode(
        x='Step',
        y='rolling_mean:Q'
    )
    
    rollmean_charts.append(run_chart + rollmean_line)

rollmean_combined_chart = None
for c in rollmean_charts:
    if rollmean_combined_chart is None:
        rollmean_combined_chart = c
    else:
        rollmean_combined_chart = alt.vconcat(rollmean_combined_chart, c)

rollmean_combined_chart

alt.VConcatChart(...)

## percent hawk by risk level

In [41]:
# for each run (= risk level), what are upper and lower values and rolling mean for % hawk?

hawkstats = []

for i in range(8) :
    # get the end of the run; we stopped at 1000 but it stabilized very early; use last 50 rounds
    # in this run, we only ran for 200 iterations, since it stabilizes quickly
    run_i = run_df[run_df.RunId == i]
    phawk_vals = run_i.percent_hawk.describe()
    # add one entry for each value with a type, so we can graph all at once in altair with a legend
    hawkstats.append({
        'risk_level': i, 
        'max': run_i.percent_hawk.max(), 
        'mean': run_i.percent_hawk.mean(), 
        'min': run_i.percent_hawk.min()
    })

hawkstats_df = pd.DataFrame(hawkstats)
hawkstats_df

,risk_level,max,mean,min
0,0,0.6425,0.458162,0.1950
1,1,0.9900,0.588259,0.1700
2,2,1.0000,0.550587,0.3225
3,3,0.9725,0.518764,0.1425
4,4,0.8625,0.480445,0.0625
5,5,0.7450,0.444648,0.0000
6,6,0.8150,0.408027,0.0025
7,7,0.7975,0.355592,0.0025


In [42]:
# for each run (= risk level), what are upper and lower values and rolling mean for % hawk?
# format in a way we can easily graph together with altair

alt_hawkstats = []

for i in range(8) :
    # get the end of the run; we stopped at 1000 but it stabilized very early; use last 50 rounds
    # ran for 200 rounds; omit first 50 before it stabilized
    run_i = run_df[(run_df.RunId == i) & (run_df.Step > 50)]
    phawk_vals = run_i.percent_hawk.describe()
    # add one entry for each value with a type, so we can graph all at once in altair with a legend
    alt_hawkstats.extend([
        {'risk_level': i, 'value': run_i.percent_hawk.max(), 'type': 'max'},
        {'risk_level': i, 'value': run_i.percent_hawk.mean(), 'type': 'mean'},
        {'risk_level': i, 'value': run_i.percent_hawk.min(), 'type': 'min'},
    ])

alt_hawkstats_df = pd.DataFrame(alt_hawkstats)
alt_hawkstats_df

,risk_level,value,type
0,0,0.545000,max
1,0,0.458242,mean
2,0,0.352500,min
3,1,0.987500,max
4,1,0.587913,mean
5,1,0.170000,min
6,2,1.000000,max
7,2,0.550361,mean
8,2,0.325000,min
9,3,0.970000,max


In [43]:
alt.Chart(alt_hawkstats_df).mark_line().encode(
    x='risk_level:N', 
    y='value',
    color=alt.Color('type').scale(domain=['min', 'mean', 'max'], range=['purple', 'blue', 'orange'])
).properties(
    title='% hawk by risk level (min, max, mean)',
    width=500,
    height=400
)

alt.Chart(...)

## points by risk level

In [44]:
# what about points?

# get points at the last round only, so we're looking at the end state

last_round_n = df.Step.max()

last_round = df[df.Step == last_round_n]

points_mean = last_round.groupby('RunId', as_index=False).aggregate('points').mean() # : ['mean', 'sum']})
points_mean

,RunId,points
0,0,13097.9745
1,1,8924.6520
2,2,9651.1170
3,3,10107.6300
4,4,10791.9780
5,5,11506.2340
6,6,11881.3975
7,7,12573.2525
8,8,16795.5520


In [45]:

alt.Chart(points_mean).mark_bar().encode(
    x=alt.Y('RunId:N', title='risk level'),
    y=alt.Y('points', title='average points'),
).properties(
    title='average points by risk level',
    width=500,
    height=400
)

alt.Chart(...)

In [46]:
# what about min/max?

# aggregrate separately so we can graph together in altair
points_mean['type'] = 'mean'

points_min = last_round.groupby('RunId', as_index=False).aggregate('points').min()
points_min['type'] = 'min'

points_max = last_round.groupby('RunId', as_index=False).aggregate('points').max()
points_max['type'] = 'max'

points_combined = pd.concat([points_mean, points_min, points_max])

points_combined

,RunId,points,type
0,0,13097.9745,mean
1,1,8924.6520,mean
2,2,9651.1170,mean
3,3,10107.6300,mean
4,4,10791.9780,mean
5,5,11506.2340,mean
6,6,11881.3975,mean
7,7,12573.2525,mean
8,8,16795.5520,mean
0,0,7008.0000,min


In [47]:
alt.Chart(points_combined).mark_line().encode(
    x=alt.Y('RunId:N', title='risk level'),
    y=alt.Y('points', title='average points'),
    color='type'
).properties(
    title='points by risk level',
    width=500,
    height=400
)

alt.Chart(...)

In [48]:
last_round

,RunId,iteration,Step,grid_size,risk_attitudes,agent_risk_level,max_agent_points,percent_hawk,AgentID,risk_level,choice,points
399601,0,0,1000,20,single,0,21012.0,0.3525,0.0,0.0,hawk,12000.7
399602,0,0,1000,20,single,0,21012.0,0.3525,1.0,6.0,dove,12396.2
399603,0,0,1000,20,single,0,21012.0,0.3525,2.0,5.0,dove,11303.1
399604,0,0,1000,20,single,0,21012.0,0.3525,3.0,6.0,dove,11306.7
399605,0,0,1000,20,single,0,21012.0,0.3525,4.0,3.0,hawk,13485.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3600004,8,0,1000,20,single,8,16821.0,0.0000,395.0,8.0,dove,16793.4
3600005,8,0,1000,20,single,8,16821.0,0.0000,396.0,8.0,dove,16795.2
3600006,8,0,1000,20,single,8,16821.0,0.0000,397.0,8.0,dove,16795.2
3600007,8,0,1000,20,single,8,16821.0,0.0000,398.0,8.0,dove,16795.2


In [49]:
alt.Chart(last_round).mark_boxplot(extent="min-max").encode(
    alt.X("points:Q").scale(zero=False),
    alt.Y("agent_risk_level:N", title="risk level"),
).properties(
    title='range of points per agent by risk level',
    width=500,
    height=400
)


alt.Chart(...)

In [50]:
# for each run (= risk level), what are upper and lower values for individual points?

points = []

for i in range(8) :
    run_i = last_round[last_round.RunId == i]
    # add one entry for each value with a type, so we can graph all at once in altair with a legend
    points.append({
        'risk_level': i, 
        'max': run_i.points.max(), 
        'mean': run_i.points.mean(), 
        'min': run_i.points.min()
    })

points_df = pd.DataFrame(points)
points_df

,risk_level,max,mean,min
0,0,20991.0,13097.9745,7008.0
1,1,12391.7,8924.6520,5611.7
2,2,12499.5,9651.1170,6619.3
3,3,12502.1,10107.6300,7610.4
4,4,14500.5,10791.9780,8656.2
5,5,13628.0,11506.2340,9366.3
6,6,14633.3,11881.3975,9366.3
7,7,18598.2,12573.2525,10095.9
